In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools
using StaticArrays

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [15]:
meanv = datavar(:mean, NormalMeanPrecision{Float64})
precv = datavar(:precision, GammaAB{Float64})

gmpnode = GaussianMeanPrecisionNode(factorisation = SA[ SA[1], SA[2], SA[3] ])

connect!(gmpnode, :mean, meanv, 1)
connect!(gmpnode, :precision, precv, 1)

y = datavar(:y, Float64)

connect!(gmpnode, :value, y, 1)

activate!(gmpnode)

qmeanupdate = link(meanv, getbelief(meanv))
qprecupdate = link(precv, getbelief(precv))

setbelief!(meanv, NormalMeanPrecision(0.0, 0.001))
setbelief!(precv, GammaAB(0.001, 0.001))

# update!(meanv, NormalMeanPrecision(0.0, 0.001))
# update!(precv, GammaAB(0.001, 0.001))

In [16]:
realprecision = 0.01
realmean = 20

data = rand(Normal(realmean, sqrt(1.0 / realprecision)), (1000, ));

In [17]:
for i in 1:1000
    for i in 1:5
        update!(y, data[i])
    end
end

subscribe!(getbelief(meanv) |> map(Any, mean) |> take(1) |> skip_complete(), logger("meanv"))
subscribe!(getbelief(precv) |> map(Any, mean) |> take(1) |> skip_complete(), logger("precv"))

[meanv] Data: 22.455096661591238
[precv] Data: 0.006054180518264514


SwitchMapSubscription()